<a href="https://colab.research.google.com/github/josegallardo-da/project-data-extraction/blob/master/Project_Data_Extraction_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [291]:
import requests
import json
import pandas as pd
import time 
import datetime

In [256]:
def query(dicc):

  dicc = json.dumps(dicc)
  remove = '{}" '

  for r in remove:
    dicc = dicc.replace(r, '').replace(':', '=').replace(',','&')
    query = dicc

  return query

In [257]:
def months(years, period=None):

  keys = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY',\
          'AUGUST', 'SEPTEMBER', 'OCTOBER',  'NOVEMBER', 'DECEMBER']
  values = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  months = {keys[i]:values[i] for i in range(len(keys))}

  if period==None:
    return [str(years)+month for month in months.values()]

  if type(period) == str:
    return str(years)+months[period]

In [259]:
def dicc_zones():
    
    zones = requests.get('https://comtrade.un.org/Data/cache/partnerAreas.json')
    zones = zones.json()['results']
    
    values = []
    
    for zone in zones:
      for (k,v) in zone.items():
        values.append(v)
    
    values = iter(values)
    id_zone_dicc = dict(zip(values, values))

    return id_zone_dicc

In [261]:
def all_query(time=list, zone='484'):

  period_years = list(range(1990, 2010))
  period_months = list(range(2010, 2021))

  check_years = all(item in period_years for item in time)
  check_months =  all(item in period_months for item in time)

  if check_months == True:
    
    #time_in_months = list(map(months, time))
    #year_months = [month for time in time_in_months for month in time]
    #partners = list(dicc_zones().keys())

    list_of_queries = []
    
    #for partner in partners:

    for year in time:
      #for ym in year_months:

      keys = ['type', 'freq', 'px', 'ps', 'r', 'p', 'rg', 'cc']
      values = ['C', 'M', 'HS', str(year), zone, 'all', 'all', 'TOTAL']

      query_dict = {keys[i]:values[i] for i in range(len(keys))}
      query_final = query(query_dict)

      list_of_queries.append(query_final)

    return list_of_queries

  if check_years == True:

    list_of_queries = []

    for year in time:

      keys = ['type', 'freq', 'px', 'ps', 'r', 'p', 'rg', 'cc']
      values = ['C', 'A', 'HS', str(year), zone, 'all', 'all', 'TOTAL']

      query_dict = {keys[i]:values[i] for i in range(len(keys))}
      query_final = query(query_dict)

      list_of_queries.append(query_final)

    return list_of_queries

In [272]:
def api_getting(api=str, queries=list):
  
  dfs = []

  for query in queries:
    response = requests.get(f'{api}{query}')
    time.sleep(1)
    data = pd.DataFrame(response.json()['dataset'])
    dfs.append(data)

  return dfs

In [273]:
def data_extraction():

  years = list(range(2010, 2021))

  final_queries = all_query(years)

  api = 'http://comtrade.un.org/api/get?'

  results_dfs = api_getting(api, final_queries)

  return results_dfs

In [274]:
trade_data = data_extraction()

In [277]:
trade_data[0]

,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,rt3ISO,ptCode,ptTitle,pt3ISO,ptCode2,ptTitle2,pt3ISO2,cstCode,cstDesc,motCode,motDesc,cmdCode,cmdDescE,qtCode,qtDesc,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
0,HS,2010,201008,August 2010,0,0,1,Imports,484,Mexico,None,116,Cambodia,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,5887887,None,None,0
1,HS,2010,201008,August 2010,0,0,2,Exports,484,Mexico,None,344,"China, Hong Kong SAR",None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,33708650,None,None,0
2,HS,2010,201008,August 2010,0,0,1,Imports,484,Mexico,None,710,South Africa,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,82538966,None,None,0
3,HS,2010,201003,March 2010,0,0,1,Imports,484,Mexico,None,348,Hungary,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,31778856,None,None,0
4,HS,2010,201003,March 2010,0,0,2,Exports,484,Mexico,None,620,Portugal,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,2216020,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4362,HS,2010,201001,January 2010,0,0,1,Imports,484,Mexico,None,40,Austria,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,38406837,None,None,0
4363,HS,2010,201001,January 2010,0,0,1,Imports,484,Mexico,None,760,Syria,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,3385,None,None,0
4364,HS,2010,201001,January 2010,0,0,1,Imports,484,Mexico,None,818,Egypt,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,29189852,None,None,0
4365,HS,2010,201001,January 2010,0,0,1,Imports,484,Mexico,None,12,Algeria,None,None,,,,,,,TOTAL,All Commodities,None,None,None,,None,None,None,None,287,None,None,0


In [276]:
trade_data[0].columns

Index(['pfCode', 'yr', 'period', 'periodDesc', 'aggrLevel', 'IsLeaf', 'rgCode',
       'rgDesc', 'rtCode', 'rtTitle', 'rt3ISO', 'ptCode', 'ptTitle', 'pt3ISO',
       'ptCode2', 'ptTitle2', 'pt3ISO2', 'cstCode', 'cstDesc', 'motCode',
       'motDesc', 'cmdCode', 'cmdDescE', 'qtCode', 'qtDesc', 'qtAltCode',
       'qtAltDesc', 'TradeQuantity', 'AltQuantity', 'NetWeight', 'GrossWeight',
       'TradeValue', 'CIFValue', 'FOBValue', 'estCode'],
      dtype='object')

In [305]:
def data_prettifier(dfs=list):

  dfs_copies = dfs.copy()
  new_dfs = []

  for df in dfs_copies:
    
    df = df.drop(['pfCode', 'yr', 'periodDesc', 'aggrLevel', 'IsLeaf', 'rgCode',\
                  'rtCode', 'rt3ISO', 'ptCode', 'pt3ISO',\
                  'ptCode2', 'ptTitle2', 'pt3ISO2', 'cstCode', 'cstDesc', 'motCode',\
                  'motDesc', 'cmdCode', 'qtCode', 'qtDesc', 'qtAltCode',\
                  'qtAltDesc', 'TradeQuantity', 'AltQuantity', 'NetWeight', 'GrossWeight',\
                  'CIFValue', 'FOBValue', 'estCode'], axis = 1) 
    
    df.columns = ['Period YYYYMM', 'Regimen', 'Country', 'Partner','Commodity', 'Trade Value']
    
    df[['Period YYYYMM']] = df[['Period YYYYMM']].applymap(str).applymap(lambda s: "{}/{}".format(s[0:4], s[4:6]))

    df[['Trade Value']] = df[['Trade Value']].astype(float)

    new_dfs.append(df)

  return new_dfs


In [306]:
new_data = data_prettifier(trade_data)
new_data[0] 

,Period YYYYMM,Regimen,Country,Partner,Commodity,Trade Value
0,2010/08,Imports,Mexico,Cambodia,All Commodities,5887887.0
1,2010/08,Exports,Mexico,"China, Hong Kong SAR",All Commodities,33708650.0
2,2010/08,Imports,Mexico,South Africa,All Commodities,82538966.0
3,2010/03,Imports,Mexico,Hungary,All Commodities,31778856.0
4,2010/03,Exports,Mexico,Portugal,All Commodities,2216020.0
...,...,...,...,...,...,...
4362,2010/01,Imports,Mexico,Austria,All Commodities,38406837.0
4363,2010/01,Imports,Mexico,Syria,All Commodities,3385.0
4364,2010/01,Imports,Mexico,Egypt,All Commodities,29189852.0
4365,2010/01,Imports,Mexico,Algeria,All Commodities,287.0


In [ ]:
def data_files(clean_dfs=list):
  concatenated_df = pd.concat(clean_dfs)
  
